# Reference:
* [Introduction](https://www.tensorflow.org/guide/low_level_intro)
* [Graphs and Sessions](https://www.tensorflow.org/guide/graphs)

# Test Environment
* TensorFlow: 1.13.0
* Python: 3.6.6

In [ ]:
import numpy as np
import tensorflow as tf

# Tensor Values
* The central unit of data in TensorFlow is the tensor.  
* A tensor's rank is its number of dimensions, while its shape is a tuple of integers specifying the array's length along each dimension.
* TensorFlow uses numpy arrays to represent tensor values.

In [ ]:
3.                                      # a rank 0 tensor; a scalar with shape [],
[1., 2., 3.]                            # a rank 1 tensor; a vector with shape [3]
[[1., 2., 3.], [4., 5., 6.]]            # a rank 2 tensor; a matrix with shape [2, 3]
[[[1., 2., 3.]], [[7., 8., 9.]]]        # a rank 3 tensor with shape [2, 1, 3]

# Basic operation: constant
* The default data type in TensorFlow is tf.float32
* Notice that printing the tensors does not output the values 3.0, 4.0, and 7.0 as you might expect. The above statements only build the computation graph. These tf.Tensor objects just represent the results of the operations that will be run.
* Each operation in a graph is given a unique name. This name is independent of the names the objects are assigned to in Python. Tensors are named after the operation that produces them followed by an output index, as in "add:0" below.

In [ ]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) # also tf.float32 implicitly
total = a + b

print(a)
print(b)
print(total)

# Session

In [ ]:
sess = tf.Session()
print(sess.run(total))
print(sess.run({'ab':(a, b), 'total':total}))

In [ ]:
# The result shows a different random value on each call to run, 
# but a consistent value during a single run (out1 and out2 receive the same random input)

vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

# Feeding
## Placeholder
* A graph can be parameterized to accept external inputs, known as `placeholders`. A placeholder is a promise to provide a value later, like a function argument.
* We can evaluate this graph with multiple inputs by using the `feed_dict` argument of the tf.Session.run method to feed concrete values to the placeholders.

In [ ]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y
print(sess.run(z, feed_dict={x: 3, y: 4.5}))
print(sess.run(z, feed_dict={x: [1, 3], y: [2, 4]}))

## Datasets
* Placeholders work for simple experiments, but tf.data are the preferred method of streaming data into a model.
* To get a runnable tf.Tensor from a Dataset you must first convert it to a tf.data.Iterator, and then call the Iterator's tf.data.Iterator.get_next method.
* Reaching the end of the data stream causes Dataset to throw an tf.errors.OutOfRangeError.

In [ ]:
# The simplest way to create an Iterator is with the tf.data.Dataset.make_one_shot_iterator method.

my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()
while True:
  try:
    print(sess.run(next_item))
  except tf.errors.OutOfRangeError:
    break

In [ ]:
# If the Dataset depends on stateful operations you may need to initialize the iterator before using it

r = tf.random_normal([10,3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer)
while True:
  try:
    print(sess.run(next_row))
  except tf.errors.OutOfRangeError:
    break

# Layer

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 3])

# linear_model = tf.layers.Dense(units=1)
# y = linear_model(x)

# Layer Function shortcuts
y = tf.layers.dense(x, units=1)

init = tf.global_variables_initializer()
sess.run(init)
print(sess.run(y, {x: [[1, 2, 3],[4, 5, 6]]}))

# Training

* Untraining case

In [ ]:
# y = -x+1

# Define the data
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
print("x.shape:", x.shape)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

# Define the model, build a simple linear model, with 1 output
linear_model = tf.layers.Dense(units=1)
y_pred = linear_model(x)

# Evaluate the predictions
# The model hasn't yet been trained, so the four "predicted" values aren't very good
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
print("Predict:", sess.run(y_pred))

# Loss
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)
print("Loss:", sess.run(loss))

* Training case

In [ ]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
for i in range(100):
  _, loss_value = sess.run((train, loss))
  print(loss_value)

print("Predict:", sess.run(y_pred))